In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import lmfit
import numpy as np
from astropy.io import ascii
from astropy import units as u
from lmfit.models import PolynomialModel, GaussianModel
from lmfit import Model as mod
from lmfit import minimize, Parameters, report_fit
import ccdproc as ccdp
from pathlib import Path
import os
import glob
import shutil

In [2]:
# os.makedirs("plots", exist_ok=True)
# os.makedirs("reports", exist_ok=True)

'''Only run these once to make file space for the graphs & data'''

In [15]:
# Load all files matching pattern
file_list = sorted(glob.glob("OHdata/*67_ii.ascii"))  # Assumes files are in the same folder as this script

# Process each file
for filename in file_list:
    filename_base = os.path.splitext(os.path.basename(filename))[0]
    data = np.loadtxt(filename, skiprows = 3)
    x = data[:, 0]
    y = data[:, 1]

    ### Polynomial Baseline Fit ###
    poly_model = PolynomialModel(degree=4)
    gooddata1 = list(np.where((x >= -30) & (x <= 0))[0])
    gooddata2 = list(np.where((x >= 10) & (x <= 30))[0])
    gooddata = gooddata1 + gooddata2

    params = poly_model.guess(y[gooddata], x=x[gooddata])
    result = poly_model.fit(y[gooddata], params, x=x[gooddata])
    y_eval = poly_model.eval(result.params, x=x)

    ### Plotting Original with Baseline ###
    # plt.figure(figsize=(7, 5))
    # plt.xlim([-30, 30])
    # plt.ylim([-.50, .50])
    # plt.plot(x, y, label='Data')
    # plt.plot(x, y_eval, label='Polynomial Baseline', color='red', linestyle='--')
    # plt.xlabel('Radio(km/s)')
    # plt.ylabel('TA(I)')
    # plt.title(f'Baseline of OHData; {filename[7:-4]}')
    # plt.legend()
    # plt.tight_layout()
    # filename_base = os.path.splitext(os.path.basename(filename))[0]
    # plt.savefig(f"plots/baseline_{filename_base}.png")
    # plt.close()

    ### Subtract Baseline ###
    y_sub = y - y_eval

    ### Plotting Subtracted Data ###
    # plt.figure(figsize=(7, 5))
    # plt.xlim([-30, 30])
    # plt.ylim([-.50, .50])
    # plt.axhline(0, color='green')
    # plt.plot(x, y, label='Data')
    # plt.plot(x, y_sub, label='Subtracted Polynomial Baseline', color='red', linestyle='--')
    # plt.xlabel('Radio(km/s)')
    # plt.ylabel('TA(I)')
    # filename_base = os.path.splitext(os.path.basename(filename))[0]
    # plt.title(f'Subtracted Baseline of OHData; {filename_base}')
    # plt.legend()
    # plt.tight_layout()
    # filename_base = os.path.splitext(os.path.basename(filename))[0]
    # plt.savefig(f"plots/subtracted_baseline_{filename_base}.png")
    # plt.close()
    
    ### Gaussian Regions ###
    region = np.where((x >= 0) & (x <= 10))[0]

    ### Plot Selected Regions (guess) ###
    # plt.figure(figsize=(7, 5))
    # plt.plot(x, y_sub, label='Data', color='grey')
    # plt.plot(x[region], y_sub[region], '-', label='Gaussian Fit', color='r')
    # plt.xlim([-50, 50])
    # plt.ylim([-.05, .05])
    # plt.xlabel('Radio(km/s)')
    # plt.ylabel('TA(I)')
    # filename_base = os.path.splitext(os.path.basename(filename))[0]
    # plt.title(f'Gaussian of OHData; {filename_base}')
    # plt.legend()
    # plt.tight_layout()
    # filename_base = os.path.splitext(os.path.basename(filename))[0]
    # plt.savefig(f"plots/gaussian_regions_{filename_base}.png")
    # plt.close()

    ### Gaussian Fits ###
    gauss_model = GaussianModel()
    params = gauss_model.guess(y_sub[region], x=x[region])
    fit = gauss_model.fit(y_sub[region], params, x=x[region])
    y_fit = gauss_model.eval(fit.params, x=x)

    ### Plotting Gaussian###
    plt.figure(figsize=(7, 5))
    plt.axhline(0, color='k')
    plt.plot(x, y_sub, label='Data', color='grey')
    plt.plot(x, y_fit, '-', label='Gaussian', color='r')
    plt.xlim([-15, 15])
    plt.ylim([-.1, .5])
    plt.xlabel('Radio(km/s)')
    plt.ylabel('TA(I)')
    plt.title(f'Gaussian of OHData; {filename_base}')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'OHData/OHgaussianplots/gaussian_fits_{filename_base}.png')
    plt.close()

    # ## Save Fit Reports ###
    # filename_base = os.path.splitext(os.path.basename(filename))[0]
    # with open(f"reports/fit_report_{filename_base}.txt", 'w') as f:
    #     f.write("\n=== Gaussian Fit ===\n")
    #     f.write(fit.fit_report())
        # f.write("\n=== Gaussian Fit 2 ===\n")
        # f.write(fit2.fit_report())
        # f.write("\n=== Gaussian Fit 3 ===\n")
        # f.write(fit3.fit_report())

print("All 36 files processed.")

All 36 files processed.


In [14]:
#print(file_list)

In [24]:
print(os.getcwd())

C:\Users\dhami\Research\STARTastro\DhamiBusch-Research25


In [40]:
# Better fit of Gaussian
def gaussFit(vel, ta, bounds):
    '''
    Purpose:
    This function fits 4 features in a single spectrum, the features assumed to be the Local arm, inter arm, perseus arm, and outer arm. Adjust the following according to features needed.
    
    Input:
    vel -- x axes, velocity
    ta -- y axes, antenna temperature
    bounds -- an array of bounds for np.where commands later in the function to setup fit ranges.
    n = number of gaussian components
    
    Output:
    fitResult -- an array of 3 fit results from LMFIT for a single gaussian
    '''
    x=vel 
    y_sub=ta
    # Set bounds:
    region = np.where((x >= bounds[0]) & (x <= bounds[1]))[0]
    # Initial Guess of the parameters
    params = gauss_model.guess(y_sub[region], x=x[region])
    # Actual best fit to the data, given initial guess
    regionfit= gauss_model.fit(y_sub[region], params, x=x[region])
    #fitresult = [region1fit, region2fit, region3fit]
    fitresult = [regionfit]
    return fitresult

In [41]:
fitResult = gaussFit(x, y_sub, bounds = [-30, 30])

In [42]:
fitResult[0].params

name,value,standard error,relative error,initial value,min,max,vary,expression
amplitude,-1.3807e-04,5972.17218,(4325563507.43%),0.949233401713168,-inf,inf,True,
center,515.709636,8.4010e+09,(1629017042.49%),7.205548013208404,-inf,inf,True,
sigma,546.333065,3.8750e+09,(709279164.26%),9.877750078781249,0.00000000,inf,True,
fwhm,1286.51603,9.1250e+09,(709279165.04%),23.260323440515663,-inf,inf,False,2.3548200*sigma
height,-1.0082e-07,5.07603352,(5034800331.17%),0.038337612664421566,-inf,inf,False,"0.3989423*amplitude/max(1e-15, sigma)"


In [39]:
# Computing the Integral
def integral(vel, ta, bounds, deltav):
    mask = (vel >= bounds[0]) & (vel <= bounds[1])
    deltav = vel[1]-vel[0]
    return np.sum(ta[mask])*deltav
# Set the path to your ASCII files
file_list = sorted(glob.glob("OHdata/*.ascii"))  # Assumes files are in the same folder as this script
bounds = [-15, 15]
# Process each file
for filename in file_list:
    data = np.loadtxt(filename, skiprows=3)
    vel = data[:, 0]
    ta = data[:, 1]
    # Gaussian subtraction
    gauss_model = GaussianModel()
    region = np.where((vel >= -30) & (vel <= 30))[0]
    params = gauss_model.guess(ta[region], x=vel[region])
    fit = gauss_model.fit(ta[region], params, x=vel[region])
    y_fit = gauss_model.eval(fit.params, x=vel)
    ta = y - y_eval
    # Compute integral
    result = integral(vel, ta, bounds, deltav)
    print(f"{filename}: Integral from {bounds[0]} to {bounds[1]} = {result}")

OHdata\158375n19250_65_ii.ascii: Integral from -15 to 15 = -9773.073961501195
OHdata\158375n19250_67_ii.ascii: Integral from -15 to 15 = 0.022257950160305177
OHdata\158500n19250_65_ii.ascii: Integral from -15 to 15 = -9773.021302655663
OHdata\158500n19250_67_ii.ascii: Integral from -15 to 15 = 0.022257830230870326
OHdata\158500n19375_65_ii.ascii: Integral from -15 to 15 = -9773.017394325432
OHdata\158500n19375_67_ii.ascii: Integral from -15 to 15 = 0.02225782132889601
OHdata\158625n19375_65_ii.ascii: Integral from -15 to 15 = -9772.998179734634
OHdata\158625n19375_67_ii.ascii: Integral from -15 to 15 = 0.02388354100382634
OHdata\158750n19375_65_ii.ascii: Integral from -15 to 15 = -10026.315488817112
OHdata\158750n19375_67_ii.ascii: Integral from -15 to 15 = 0.023883525938040828
OHdata\158750n19500_65_ii.ascii: Integral from -15 to 15 = -10026.311113444557
OHdata\158750n19500_67_ii.ascii: Integral from -15 to 15 = 0.023883515516992113
OHdata\158875n19500_65_ii.ascii: Integral from -15 t

In [25]:
# # Specify the current directory name and the new name
# current_name = "plots"
# new_name = "OHgaussianplots"
# # Rename the directory
# os.rename(current_name, new_name)
# print(f"Directory renamed from '{current_name}' to '{new_name}'")

Directory renamed from 'plots' to 'OHgaussianplots'


In [30]:
# # Source directory path
# source_dir = 'C:/Users/dhami/Research/STARTastro/DhamiBusch-Research25/OHgaussianplots'
# # Destination directory path
# destination_dir = 'C:/Users/dhami/Research/STARTastro/DhamiBusch-Research25/OHData'
# # Move the directory
# shutil.move(source_dir, destination_dir)

'C:/Users/dhami/Research/STARTastro/DhamiBusch-Research25/OHData\\OHgaussianplots'